In [28]:
import pandas as pd
import error_prompts as p
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd

# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()



prompt_name = 'error_class_LLM'
prompt = p.error_class_LLM

file = "structured_unstructured_singleterm_dynamic_fewshot_4-turbo-preview"

DATA_PATH = f'../../Datasets/Evaluations/Sentiment Analysis/Error_Analysis/structured_unstructured/{file}.csv'
OUTPUT_PATH = f"../../Datasets/Evaluations/Sentiment Analysis/Error_Analysis/error_class_LLM_analysis.csv"

MODEL = "gpt-4-turbo-preview"
TEMP = 0

struct_unstruct = 'unstructured_analysis'

In [29]:
import os
df = pd.read_csv(DATA_PATH)

df_correct = df[df['error'] == 'correct']
df_correct_samples = df_correct.sample(n=5, random_state=42)

df_error = df[df['error'] == 'error']
df_error_samples = df_error.sample(n=20, random_state=42)

print(df_correct_samples.shape, df_error_samples.shape)
print(df_correct_samples.head())
print(df_error_samples.head())

if os.path.exists(OUTPUT_PATH):
    df_answer = pd.read_csv(OUTPUT_PATH)
else:
    df_answer = pd.DataFrame(columns=['Prompt_Name', 'Prompt', 'Model', 'Database', 'Answer', 'Context'])





(5, 22) (20, 22)
    Unnamed: 0.1  Unnamed: 0  text_id  term_id  \
63           317         317      121      317   
89           205         205       79      205   
80           423         423      159      423   
95           135         135       52      135   
67           359         359      137      359   

                                                 text         term  polarity  \
63  Once at the table our waitress managed on two ...     waitress  negative   
89  I like the somosas, chai, and the chole, but t...         chai  positive   
80  I understand the area and folks you need not c...        foods  positive   
95  The service was polite, friendly and prompt, t...      service  positive   
67  The food is decent, average bistro food, the d...  bistro food   neutral   

    example1_term_id                                    example1_prompt  \
63             10492  Input: "At our table we didn't get any bread u...   
89              2517  Input: "Theme and actors are 

In [30]:
def execute(df, prompt, df_answer):
    i = 1
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, timeout=150)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, timeout=150)
    examples = ""
    examples += "False Decisions:\n"
    for j in range(len(df_error_samples)):
        examples += f"Task {i}:\n"
        examples += "Prompt: " + df_error_samples.iloc[j]['prompt'] + "\n"
        examples += "Sentiment Decision: " + str(df_error_samples.iloc[j]['polarity_pred']) + "\n"
        examples += "Explanation: " + df_error_samples.iloc[j][struct_unstruct] + "\n"
        examples += "Actual Sentiment: " + str(df_error_samples.iloc[j]['polarity']) + "\n"
        i += 1
    examples += "\nCorrect Decisions (Only as a reference):\n"
    for j in range(len(df_correct_samples)):
        examples += f"Task {i}:\n"
        examples += "Prompt: " + df_correct_samples.iloc[j]['prompt'] + "\n"
        examples += "Sentiment Decision: " + str(df_correct_samples.iloc[j]['polarity_pred']) + "\n"
        examples += "Explanation: " + df_correct_samples.iloc[j][struct_unstruct] + "\n"
        examples += "Actual Sentiment: " + str(df_correct_samples.iloc[j]['polarity']) + "\n"
        i += 1
    chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
    try: 
        result = chain.run(examples = examples, callbacks=[handler])
    except Exception as e:
        print(e)
        print(chain.prompt.format_prompt(examples = examples).to_string())
    handler.langfuse.flush()
    prompt_text = chain.prompt.format_prompt(examples = examples).to_string()
    print(prompt_text)
    print("\n")
    print(result)
    print("\n")
    new_row = {'Prompt_Name': prompt_name, 'Prompt': prompt_text, 'Model': MODEL, 'Database': file, 'Answer': result, 'Context': struct_unstruct}
    df_new_row = pd.DataFrame([new_row])
    df_answer = pd.concat([df_answer, df_new_row])
    return result, df_answer


answer, df_answer = execute(df, prompt, df_answer)
    



In the following I will give you a few Aspect Based Sentiment Analysis tasks together with a sentiment decision, details about the decision and the actual sentiment towards each term.
The Prediction was made by an LLM.
Can you please group the wrong decisions into distinct fault categories? Please also indicate how often each one occurs.
There are also some correct decisions in the examples. Please just use them as a reference and don't categorize them.
False Decisions:
Task 1:
Prompt: Example 1:
Input: "If you are looking for service and all the frills $$$ - Il Mulino is the best for both - if you are looking for a fabulous dinner in/out - this restaurant is it!"
Term: service
Output: neutral


Example 2:
Input: "Don't know about the rest of the food, but the burgers are one of New York's hidden treats."
Term: burgers
Output: positive


Example 3
Input: "The rest of the meal was good, but a little over priced for Chinese take out style food."
Term: priced
Output: negative

    
Task:


In [31]:
print(df_answer)

       Prompt_Name                                             Prompt  \
0   error_class_ML  In the following I will give you a few Aspect ...   
1  error_class_LLM  In the following I will give you a few Aspect ...   
2  error_class_LLM  In the following I will give you a few Aspect ...   
3  error_class_LLM  In the following I will give you a few Aspect ...   
4  error_class_LLM  In the following I will give you a few Aspect ...   
5  error_class_LLM  In the following I will give you a few Aspect ...   
0  error_class_LLM  In the following I will give you a few Aspect ...   

                 Model                                           Database  \
0  gpt-4-turbo-preview                                logistic_regression   
1  gpt-4-turbo-preview  structured_unstructured_singleterm_zeroshot_gp...   
2  gpt-4-turbo-preview  structured_unstructured_singleterm_zeroshot_gp...   
3  gpt-4-turbo-preview  structured_unstructured_singleterm_fix_fewshot...   
4  gpt-4-turbo-preview  struct

In [27]:
df_answer.to_csv(OUTPUT_PATH, index=False)